# Preparing data

This notebook reads data processed by Galaxy and

 1. Validates sites to ensire that coordinates are correct
 2. Adds information about variants of concern (VOC) and sites under selection


In [ ]:
import pandas as pd

In [ ]:
!pip install biopython pandasql

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
funclass_translation = {'SILENT':'Synonymous','MISSENSE':'Non-synonymous','NONSENSE':'Stop','.':'Non-coding','NONE':'Non-coding'}

## Which dataset to run notebook on?
At this time there are three possible datasets:

 - Boston: `bos`
 - COG-Pre: `cog-pre`
 - COG-Post: `cog-post`

Setting this variable runs all notebook content for this particular dataset. The actual paths are pulled out from `datasets` dict (next cell)

In [ ]:
dataset = 'bos'

In [ ]:
datasets = {
    'bos':
        [   
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz',
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_var.tsv.gz'
        ],
    'cog-pre': 
        [   
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20200917_by_sample.tsv.gz',
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20200917_by_var.tsv.gz'
        ],
    'cog-post': 
        [   
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20201120_by_sample.tsv.gz',
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20201120_by_var.tsv.gz'
        ]
}

In [ ]:
# SARS-CoV-2 genome assembly url
gnm_url = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009/858/895/GCF_009858895.2_ASM985889v3/GCF_009858895.2_ASM985889v3_genomic.gbff.gz'
gnm_file = gnm_url.split('/')[-1]

# VOC data
voc_url = 'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/voc/voc.tsv.gz'

# Selection data
sel_url = 'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/selection/selection.tsv.gz'

In [ ]:
# Get SARS-CoV-2 RefSeq genomes (in GenBank format) from NCBI
import os.path
from os import path
if not path.exists(gnm_file[:-3]):
    !wget -nc {gnm_url}
    !gunzip {gnm_file}
else:
    print('File {} is already here\nDoing nothing!'.format(gnm_file))

In [ ]:
from Bio import SeqIO
genome = SeqIO.read(gnm_file[0:len(gnm_file)-3], "genbank")

In [ ]:
# Get variants by sample
var = pd.read_csv(datasets[dataset][0],sep='\t')

In [ ]:
len(var)

In [ ]:
var = var.replace({'FUNCLASS':funclass_translation})

In [ ]:
# Changing coordinates to 0-based
var['POS'] = var['POS']-1

In [ ]:
# Validation function for checking against genome

def check_against_genome(seqobject,df,pos_base_list,l):
    wrong = []
    slip_sites = dict()
    bad = 0
    good = 0
    pb = df[pos_base_list].to_numpy()
    for item in pb:
        base = seqobject[item[0]:(item[0])+l].seq
        if base != item[1] and len(base) == len(item[1]):
            slip = seqobject[(item[0]-1):(item[0]-1)+l].seq
            if slip == item[1]:
                good += 1
                slip_sites[item[0]] = True
            else:
                bad += 1
                wrong.append([item[1],base,item[0]])
        elif base == item[1] and len(base) == len(item[1]): 
            good += 1
    print ('Total = {}, Wrong = {}, Correct = {}'.format(len(pb),bad,good))
    return(slip_sites,wrong)

In [ ]:
check_against_genome(genome,var,['POS','REF'],1)

In [ ]:
var.head()

In [ ]:
# Variants by site
sites = pd.read_csv(datasets[dataset][1],sep='\t')

In [ ]:
len(sites)

In [ ]:
sites = sites.replace({'FUNCLASS':funclass_translation})

In [ ]:
# Changing coordinates to 0-based
sites['POS'] = sites['POS']-1

In [ ]:
check_against_genome(genome,sites,['POS','REF'],1)

In [ ]:
sites.head()

In [ ]:
sel = pd.read_csv(sel_url,sep='\t')

In [ ]:
# Add info about sites under sleection to the main variant table
sites = pysqldf('select sites.*, sel.fel,sel.meme,sel.freq from sites left join sel on sites.POS >= sel.pos and sites.POS <= sel.pos+2 ')

In [ ]:
sites.head()

In [ ]:
voc = pd.read_csv(voc_url, sep='\t',names=['voc_set','mut','position'],header=None)

In [ ]:
voc.head()

In [ ]:
for item in voc['voc_set'].unique():
    sites = pysqldf('select sites.*, mut from sites left join voc on (POS >= position and POS < position+3) and voc_set = "{0}"'.format(item))
    sites = sites.rename(columns={"mut": item})

In [ ]:
pysqldf('select * from sites where P1 is not null')

In [ ]:
len(sites)

In [ ]:
# Assumes df has columns labelled 'ALT' and 'REF'
def chng_type(df):
    df.loc[df['REF'].str.len() == df['ALT'].str.len(), 'type'] = 'SNP'
    df.loc[df['REF'].str.len() != df['ALT'].str.len(), 'type'] = 'Indel'

In [ ]:
chng_type(var)
chng_type(sites)

In [ ]:
var.to_csv('{}_by_sample_processed.tsv'.format(dataset),sep='\t',index=False)
sites.to_csv('{}_by_var_processed.tsv'.format(dataset),sep='\t',index=False)
!gzip *.tsv

In [ ]:
!ls

In [ ]:
from google.colab import files
files.download('{}_by_sample_processed.tsv.gz'.format(dataset))
files.download('{}_by_var_processed.tsv.gz'.format(dataset))